In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from catboost import CatBoostClassifier
import optuna
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler

tqdm.pandas()

In [5]:
#import data
df_train = pd.read_csv('./data/train_folds.zip')
df_test=  pd.read_csv('./data/Test.zip')
submission = pd.read_csv('./data/SampleSubmission.csv')

In [9]:
train = df_train.copy()
test = df_test.copy()

cat_cols = [
    'REGION',
#     'TENURE',
]

num_cols = [
    'MONTANT',
    'FREQUENCE_RECH',
    'REVENUE',
    'ARPU_SEGMENT',
    'FREQUENCE',
    'DATA_VOLUME',
    'ON_NET', 
    'ORANGE',
    'TIGO',
    'ZONE1',
    'ZONE2',
    'REGULARITY',
    'FREQ_TOP_PACK',
    'TENURE' # !!!
]

target = 'CHURN'

mapping = {
    'D 3-6 month': 1,
    'E 6-9 month': 2,
    'F 9-12 month': 3,
    'G 12-15 month': 4,
    'H 15-18 month': 5,
    'I 18-21 month': 6,
    'J 21-24 month': 7,
    'K > 24 month': 8,
}

train['TENURE'] = train['TENURE'].map(mapping) 
test['TENURE'] = test['TENURE'].map(mapping)

train['REGION'] = train['REGION'].fillna('DAKAR')
test['REGION'] = test['REGION'].fillna('DAKAR')

# for col in tqdm(num_cols):
#     col_mean = train[col].mean()
#     train[col] = train[col].fillna(col_mean)
#     test[col] = test[col].fillna(col_mean)
#     train[col] = np.log1p(train[col])
#     test[col] = np.log1p(test[col])

In [10]:
useful_cols = [
    'REGION',
    'TENURE',
    # 'MRG',  # constant
    # 'TOP_PACK',  # wtf column
    'MONTANT',
    'FREQUENCE_RECH',
    'REVENUE',
    'ARPU_SEGMENT',
    'FREQUENCE',
    'DATA_VOLUME',
    'ON_NET', 
    'ORANGE',
    'TIGO',
    'ZONE1',
    'ZONE2',
    'REGULARITY',
    'FREQ_TOP_PACK'
]

final_predictions = []
scores = []

target = 'CHURN'

for fold in tqdm(range(5), 'folds'):
    xtrain = train[train['kfold'] != fold][useful_cols]
    ytrain = train[train['kfold'] != fold][target]
    
    xvalid = train[train['kfold'] == fold][useful_cols]
    yvalid = train[train['kfold'] == fold][target]
    
    xtest = test[useful_cols]

    model = CatBoostClassifier(
        n_estimators=1000,
        random_state=42, 
        cat_features=cat_cols
    )
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    score = roc_auc_score(yvalid, preds_valid)
    scores.append(score)
    print(fold, score)    

print(np.mean(scores), np.std(scores))

# 0.7696186855892555 0.00043098014634677365 fillna через REGION
# 0.7694106138854746 0.000588962476667447 fillna через TENURE
# 0.7701952731313458 0.0006247026357018513 fillna через mean по колонке
# 0.769901864294696 0.000610047154013374 fillna через mean по колонке & ohe for region
# 0.769901864294696 0.000610047154013374 fillna через mean по колонке & ohe for region + StandardScaler for Tenure (same as prev)
# 0.770010208315598 0.0005090535431930928 fillna через mean по колонке & ohe for region & tenure [private: 0.853185019447002]
# 0.770008209799155 0.0005120012284348264 fillna через mean по колонке & ohe for region & tenure for whole train and test datasets & StScaler on whole ds
# 0.7701761776716778 0.0007820978999800708 fillna через mean по колонке & ohe for region & tenure for whole train and test datasets & StScaler on whole ds + target encoding by reg & ten [private 0.862163874472749]
# 0.7690741534134118 0.0008824035170153271 fillna через mean по колонке & ohe for region & tenure for whole train and test datasets & StScaler on whole ds + target encoding by reg & ten + изменил фолды (!= и ==)
# 0.7690825678568933 0.0008789684650258934  fillna через mean по колонке & ohe for region & tenure for whole train and test datasets & StScaler on whole ds + target encoding by reg & ten + изменил фолды (!= и ==)
# 0.7698245780943753 0.000724931679586648 lgb - fillna через mean по колонке & ohe for region & tenure for whole train and test datasets & StScaler on whole ds + target encoding by reg & ten + изменил фолды (!= и ==)
# 0.7694104784482177 0.0013440865822247955 lgb - fillna через mean по колонке & ohe for region & tenure for whole train and test datasets & StScaler on whole ds + target encoding by reg & ten + изменил фолды (!= и ==) + optuna params [0.760814445393041]
# 0.7696972390287016 0.0009092867210997232 lgb - fillna через mean по колонке & ohe for region & tenure for whole train and test datasets & StScaler on whole ds + target encoding by reg & ten + изменил фолды (!= и ==) + num cols log1

# 0.7681898213815074 0.001549215694430003 cb + fillna через REGION
# x.xxx x.xxx cb + fillna через REGION + TENURE map as num 

folds:   0%|          | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.24823
0:	learn: 0.4718824	total: 623ms	remaining: 10m 22s
1:	learn: 0.3581121	total: 1.27s	remaining: 10m 34s
2:	learn: 0.3118517	total: 1.82s	remaining: 10m 4s
3:	learn: 0.2957341	total: 2.35s	remaining: 9m 44s
4:	learn: 0.2862375	total: 2.88s	remaining: 9m 33s
5:	learn: 0.2823673	total: 3.39s	remaining: 9m 21s
6:	learn: 0.2796395	total: 3.92s	remaining: 9m 16s
7:	learn: 0.2785295	total: 4.43s	remaining: 9m 9s
8:	learn: 0.2768836	total: 4.93s	remaining: 9m 3s
9:	learn: 0.2764747	total: 5.44s	remaining: 8m 58s
10:	learn: 0.2757857	total: 5.93s	remaining: 8m 53s
11:	learn: 0.2756125	total: 6.51s	remaining: 8m 56s
12:	learn: 0.2752015	total: 7.09s	remaining: 8m 58s
13:	learn: 0.2748788	total: 7.6s	remaining: 8m 55s
14:	learn: 0.2747598	total: 8.1s	remaining: 8m 52s
15:	learn: 0.2746749	total: 8.69s	remaining: 8m 54s
16:	learn: 0.2745337	total: 9.27s	remaining: 8m 56s
17:	learn: 0.2743708	total: 9.88s	remaining: 8m 59s
18:	learn: 0.2742821	total: 10.5s	remaining: 8m

157:	learn: 0.2717244	total: 1m 33s	remaining: 8m 18s
158:	learn: 0.2717147	total: 1m 34s	remaining: 8m 18s
159:	learn: 0.2717069	total: 1m 34s	remaining: 8m 17s
160:	learn: 0.2716959	total: 1m 35s	remaining: 8m 17s
161:	learn: 0.2716881	total: 1m 36s	remaining: 8m 16s
162:	learn: 0.2716767	total: 1m 36s	remaining: 8m 16s
163:	learn: 0.2716640	total: 1m 37s	remaining: 8m 15s
164:	learn: 0.2716538	total: 1m 37s	remaining: 8m 14s
165:	learn: 0.2716422	total: 1m 38s	remaining: 8m 14s
166:	learn: 0.2716322	total: 1m 38s	remaining: 8m 13s
167:	learn: 0.2716301	total: 1m 39s	remaining: 8m 13s
168:	learn: 0.2716186	total: 1m 40s	remaining: 8m 12s
169:	learn: 0.2716066	total: 1m 40s	remaining: 8m 11s
170:	learn: 0.2716001	total: 1m 41s	remaining: 8m 11s
171:	learn: 0.2715940	total: 1m 41s	remaining: 8m 10s
172:	learn: 0.2715846	total: 1m 42s	remaining: 8m 9s
173:	learn: 0.2715782	total: 1m 43s	remaining: 8m 9s
174:	learn: 0.2715715	total: 1m 43s	remaining: 8m 8s
175:	learn: 0.2715641	total: 1m

311:	learn: 0.2706146	total: 3m 7s	remaining: 6m 53s
312:	learn: 0.2706042	total: 3m 8s	remaining: 6m 52s
313:	learn: 0.2705973	total: 3m 8s	remaining: 6m 52s
314:	learn: 0.2705905	total: 3m 9s	remaining: 6m 51s
315:	learn: 0.2705860	total: 3m 9s	remaining: 6m 51s
316:	learn: 0.2705852	total: 3m 10s	remaining: 6m 50s
317:	learn: 0.2705775	total: 3m 10s	remaining: 6m 49s
318:	learn: 0.2705701	total: 3m 11s	remaining: 6m 48s
319:	learn: 0.2705666	total: 3m 12s	remaining: 6m 48s
320:	learn: 0.2705563	total: 3m 12s	remaining: 6m 47s
321:	learn: 0.2705542	total: 3m 13s	remaining: 6m 47s
322:	learn: 0.2705475	total: 3m 13s	remaining: 6m 46s
323:	learn: 0.2705400	total: 3m 14s	remaining: 6m 45s
324:	learn: 0.2705354	total: 3m 15s	remaining: 6m 45s
325:	learn: 0.2705315	total: 3m 15s	remaining: 6m 44s
326:	learn: 0.2705278	total: 3m 16s	remaining: 6m 44s
327:	learn: 0.2705166	total: 3m 16s	remaining: 6m 43s
328:	learn: 0.2705058	total: 3m 17s	remaining: 6m 42s
329:	learn: 0.2705044	total: 3m 1

464:	learn: 0.2697859	total: 4m 40s	remaining: 5m 23s
465:	learn: 0.2697748	total: 4m 41s	remaining: 5m 22s
466:	learn: 0.2697726	total: 4m 42s	remaining: 5m 22s
467:	learn: 0.2697694	total: 4m 42s	remaining: 5m 21s
468:	learn: 0.2697606	total: 4m 43s	remaining: 5m 21s
469:	learn: 0.2697550	total: 4m 44s	remaining: 5m 20s
470:	learn: 0.2697544	total: 4m 44s	remaining: 5m 19s
471:	learn: 0.2697506	total: 4m 45s	remaining: 5m 19s
472:	learn: 0.2697407	total: 4m 46s	remaining: 5m 18s
473:	learn: 0.2697338	total: 4m 46s	remaining: 5m 18s
474:	learn: 0.2697326	total: 4m 47s	remaining: 5m 17s
475:	learn: 0.2697248	total: 4m 48s	remaining: 5m 17s
476:	learn: 0.2697181	total: 4m 48s	remaining: 5m 16s
477:	learn: 0.2697177	total: 4m 49s	remaining: 5m 16s
478:	learn: 0.2697140	total: 4m 50s	remaining: 5m 15s
479:	learn: 0.2697091	total: 4m 50s	remaining: 5m 15s
480:	learn: 0.2697049	total: 4m 51s	remaining: 5m 14s
481:	learn: 0.2696990	total: 4m 52s	remaining: 5m 14s
482:	learn: 0.2696943	total:

...

82:	learn: 0.2728431	total: 51.1s	remaining: 9m 24s
83:	learn: 0.2728249	total: 51.7s	remaining: 9m 23s
84:	learn: 0.2728177	total: 52.2s	remaining: 9m 22s
85:	learn: 0.2727996	total: 52.9s	remaining: 9m 22s
86:	learn: 0.2727783	total: 53.6s	remaining: 9m 22s
87:	learn: 0.2727720	total: 54.2s	remaining: 9m 21s
88:	learn: 0.2727554	total: 54.8s	remaining: 9m 21s
89:	learn: 0.2727469	total: 55.4s	remaining: 9m 20s
90:	learn: 0.2727323	total: 56.1s	remaining: 9m 20s
91:	learn: 0.2727152	total: 56.6s	remaining: 9m 18s
92:	learn: 0.2726883	total: 57.2s	remaining: 9m 17s
93:	learn: 0.2726824	total: 57.8s	remaining: 9m 16s
94:	learn: 0.2726735	total: 58.3s	remaining: 9m 15s
95:	learn: 0.2726593	total: 59s	remaining: 9m 15s
96:	learn: 0.2726529	total: 59.6s	remaining: 9m 14s
97:	learn: 0.2726438	total: 1m	remaining: 9m 14s
98:	learn: 0.2726295	total: 1m	remaining: 9m 13s
99:	learn: 0.2726137	total: 1m 1s	remaining: 9m 13s
100:	learn: 0.2725976	total: 1m 2s	remaining: 9m 13s
101:	learn: 0.27258

236:	learn: 0.2713841	total: 2m 25s	remaining: 7m 47s
237:	learn: 0.2713814	total: 2m 25s	remaining: 7m 46s
238:	learn: 0.2713727	total: 2m 26s	remaining: 7m 46s
239:	learn: 0.2713671	total: 2m 27s	remaining: 7m 45s
240:	learn: 0.2713580	total: 2m 27s	remaining: 7m 45s
241:	learn: 0.2713523	total: 2m 28s	remaining: 7m 44s
242:	learn: 0.2713513	total: 2m 29s	remaining: 7m 44s
243:	learn: 0.2713513	total: 2m 29s	remaining: 7m 43s
244:	learn: 0.2713484	total: 2m 30s	remaining: 7m 42s
245:	learn: 0.2713474	total: 2m 30s	remaining: 7m 42s
246:	learn: 0.2713382	total: 2m 31s	remaining: 7m 41s
247:	learn: 0.2713373	total: 2m 32s	remaining: 7m 40s
248:	learn: 0.2713308	total: 2m 32s	remaining: 7m 40s
249:	learn: 0.2713236	total: 2m 33s	remaining: 7m 39s
250:	learn: 0.2713150	total: 2m 33s	remaining: 7m 39s
251:	learn: 0.2713114	total: 2m 34s	remaining: 7m 38s
252:	learn: 0.2713112	total: 2m 35s	remaining: 7m 38s
253:	learn: 0.2713059	total: 2m 35s	remaining: 7m 38s
254:	learn: 0.2713046	total:

389:	learn: 0.2704612	total: 4m	remaining: 6m 16s
390:	learn: 0.2704537	total: 4m 1s	remaining: 6m 16s
391:	learn: 0.2704489	total: 4m 2s	remaining: 6m 15s
392:	learn: 0.2704447	total: 4m 2s	remaining: 6m 14s
393:	learn: 0.2704358	total: 4m 3s	remaining: 6m 14s
394:	learn: 0.2704312	total: 4m 3s	remaining: 6m 13s
395:	learn: 0.2704218	total: 4m 4s	remaining: 6m 13s
396:	learn: 0.2704157	total: 4m 5s	remaining: 6m 12s
397:	learn: 0.2704083	total: 4m 5s	remaining: 6m 11s
398:	learn: 0.2704056	total: 4m 6s	remaining: 6m 11s
399:	learn: 0.2703962	total: 4m 6s	remaining: 6m 10s
400:	learn: 0.2703895	total: 4m 7s	remaining: 6m 9s
401:	learn: 0.2703836	total: 4m 8s	remaining: 6m 9s
402:	learn: 0.2703748	total: 4m 9s	remaining: 6m 8s
403:	learn: 0.2703675	total: 4m 9s	remaining: 6m 8s
404:	learn: 0.2703583	total: 4m 10s	remaining: 6m 7s
405:	learn: 0.2703501	total: 4m 10s	remaining: 6m 7s
406:	learn: 0.2703428	total: 4m 11s	remaining: 6m 6s
407:	learn: 0.2703397	total: 4m 12s	remaining: 6m 5s


543:	learn: 0.2696096	total: 5m 35s	remaining: 4m 41s
544:	learn: 0.2696064	total: 5m 36s	remaining: 4m 40s
545:	learn: 0.2696042	total: 5m 36s	remaining: 4m 40s
546:	learn: 0.2696001	total: 5m 37s	remaining: 4m 39s
547:	learn: 0.2695937	total: 5m 38s	remaining: 4m 38s
548:	learn: 0.2695887	total: 5m 38s	remaining: 4m 38s
549:	learn: 0.2695826	total: 5m 39s	remaining: 4m 37s
550:	learn: 0.2695792	total: 5m 39s	remaining: 4m 36s
551:	learn: 0.2695758	total: 5m 40s	remaining: 4m 36s
552:	learn: 0.2695706	total: 5m 41s	remaining: 4m 35s
553:	learn: 0.2695628	total: 5m 41s	remaining: 4m 35s
554:	learn: 0.2695570	total: 5m 42s	remaining: 4m 34s
555:	learn: 0.2695525	total: 5m 42s	remaining: 4m 33s
556:	learn: 0.2695471	total: 5m 43s	remaining: 4m 33s
557:	learn: 0.2695424	total: 5m 43s	remaining: 4m 32s
558:	learn: 0.2695361	total: 5m 44s	remaining: 4m 31s
559:	learn: 0.2695322	total: 5m 45s	remaining: 4m 31s
560:	learn: 0.2695269	total: 5m 45s	remaining: 4m 30s
561:	learn: 0.2695209	total:

CatBoostError: bad allocation